In [1]:
# Copyright 2020 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# NVTabular demo on Rossmann data - TensorFlow

## Overview

NVTabular is a feature engineering and preprocessing library for tabular data designed to quickly and easily manipulate terabyte scale datasets used to train deep learning based recommender systems.  It provides a high level abstraction to simplify code and accelerates computation on the GPU using the RAPIDS cuDF library.

### Learning objectives

In the previous notebooks ([rossmann-store-sales-preproc.ipynb](https://github.com/NVIDIA/NVTabular/blob/main/examples/rossmann/rossmann-store-sales-preproc.ipynb) and [rossmann-store-sales-feature-engineering.ipynb](https://github.com/NVIDIA/NVTabular/blob/main/examples/rossmann/rossmann-store-sales-feature-engineering.ipynb)), we downloaded, preprocessed and created features for the dataset. Now, we are ready to train our deep learning model on the dataset. In this notebook, we use **TensorFlow** with the NVTabular data loader for TensorFlow to accelereate the training pipeline.

In [2]:
import os
import math
import nvtabular as nvt
import glob

## Loading NVTabular workflow
This time, we only need to define our data directories. We can load the data schema from the NVTabular workflow.

In [3]:
DATA_DIR = os.environ.get("OUTPUT_DATA_DIR", "./data")
PREPROCESS_DIR = os.path.join(DATA_DIR, 'ross_pre')
PREPROCESS_DIR_TRAIN = os.path.join(PREPROCESS_DIR, 'train')
PREPROCESS_DIR_VALID = os.path.join(PREPROCESS_DIR, 'valid')

What files are available to train on in our directories?

In [4]:
!ls $PREPROCESS_DIR

train  valid


In [5]:
!ls $PREPROCESS_DIR_TRAIN

0.0b096162c7e54285961978c247e4aa27.parquet  _file_list.txt  _metadata.json
1.2b7b8b6edf4d425e87f09d2a46d15f3b.parquet  _metadata


In [6]:
!ls $PREPROCESS_DIR_VALID

_metadata  part.0.parquet


To load a saved NVTabular workflow, we need to initalize a NVTabular workflow, first.

In [7]:
# Note, we can initialize it with an empty schema
proc = nvt.Workflow(
    cat_names=[],
    cont_names=[],
    label_name=[]
)
proc.load_stats(PREPROCESS_DIR + 'stats_and_workflow')

We extract the categorical, continuous and label column names from the NVTabular workflow.

In [8]:
CATEGORICAL_COLUMNS = proc.columns_ctx['final']['cols']['categorical']
CONTINUOUS_COLUMNS = proc.columns_ctx['final']['cols']['continuous']
LABEL_COLUMNS = proc.columns_ctx['final']['cols']['label']

COLUMNS = CATEGORICAL_COLUMNS + CONTINUOUS_COLUMNS + LABEL_COLUMNS

We load the statistics for embedding tables of our neural network. The following shows the cardinality of each categorical variable along with its associated embedding size. Each entry is of the form `(cardinality, embedding_size)`.

In [9]:
EMBEDDING_TABLE_SHAPES = nvt.ops.get_embedding_sizes(proc)
EMBEDDING_TABLE_SHAPES

{'Assortment': (4, 3),
 'CompetitionMonthsOpen': (26, 10),
 'CompetitionOpenSinceYear': (24, 9),
 'Day': (32, 11),
 'DayOfWeek': (8, 5),
 'Events': (22, 9),
 'Month': (13, 7),
 'Promo2SinceYear': (9, 5),
 'Promo2Weeks': (27, 10),
 'PromoInterval': (4, 3),
 'Promo_bw': (7, 5),
 'Promo_fw': (7, 5),
 'SchoolHoliday_bw': (9, 5),
 'SchoolHoliday_fw': (9, 5),
 'State': (13, 7),
 'StateHoliday': (3, 3),
 'StateHoliday_bw': (4, 3),
 'StateHoliday_fw': (4, 3),
 'Store': (1116, 16),
 'StoreType': (5, 4),
 'Week': (53, 15),
 'Year': (4, 3)}

## Training a Network

Now that our data is preprocessed and saved out, we can leverage `dataset`s to read through the preprocessed parquet files in an online fashion to train neural networks.

We'll start by setting some universal hyperparameters for our model and optimizer. These settings will be the same across all of the frameworks that we explore in the different notebooks.

If you're interested in contributing to NVTabular, feel free to take this challenge on and submit a pull request if successful. 12% RMSPE is achievable using the Novograd optimizer, but we know of no Novograd implementation for TensorFlow that supports sparse gradients, and so we are not including that solution below.

In [10]:
EMBEDDING_DROPOUT_RATE = 0.04
DROPOUT_RATES = [0.001, 0.01]
HIDDEN_DIMS = [1000, 500]
BATCH_SIZE = 65536
LEARNING_RATE = 0.001
EPOCHS = 25

# TODO: Calculate on the fly rather than recalling from previous analysis.
MAX_SALES_IN_TRAINING_SET = 38722.0
MAX_LOG_SALES_PREDICTION = 1.2 * math.log(MAX_SALES_IN_TRAINING_SET + 1.0)

TRAIN_PATHS = sorted(glob.glob(os.path.join(PREPROCESS_DIR_TRAIN, '*.parquet')))
VALID_PATHS = sorted(glob.glob(os.path.join(PREPROCESS_DIR_VALID, '*.parquet')))

## TensorFlow
<a id="TensorFlow"></a>


### TensorFlow: Preparing Datasets

`KerasSequenceLoader` wraps a lightweight iterator around a `dataset` object to handle chunking, shuffling, and application of any workflows (which can be applied online as a preprocessing step). For column names, can use either a list of string names or a list of TensorFlow `feature_columns` that will be used to feed the network

In [11]:
import tensorflow as tf

# we can control how much memory to give tensorflow with this environment variable
# IMPORTANT: make sure you do this before you initialize TF's runtime, otherwise
# it's too late and TF will have claimed all free GPU memory
os.environ['TF_MEMORY_ALLOCATION'] = "8192" # explicit MB
os.environ['TF_MEMORY_ALLOCATION'] = "0.5" # fraction of free memory
from nvtabular.loader.tensorflow import KerasSequenceLoader, KerasSequenceValidater

# cheap wrapper to keep things some semblance of neat
def make_categorical_embedding_column(name, dictionary_size, embedding_dim):
    return tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_identity(name, dictionary_size),
        embedding_dim
    )

# instantiate our columns
categorical_columns = [
    make_categorical_embedding_column(name, *EMBEDDING_TABLE_SHAPES[name]) for
        name in CATEGORICAL_COLUMNS
]
continuous_columns = [
    tf.feature_column.numeric_column(name, (1,)) for name in CONTINUOUS_COLUMNS
]

# feed them to our datasets
train_dataset_tf = KerasSequenceLoader(
    TRAIN_PATHS, # you could also use a glob pattern
    feature_columns=categorical_columns+continuous_columns,
    batch_size=BATCH_SIZE,
    label_names=LABEL_COLUMNS,
    shuffle=True,
    buffer_size=0.06 # amount of data, as a fraction of GPU memory, to load at once
)

valid_dataset_tf = KerasSequenceLoader(
    VALID_PATHS, # you could also use a glob pattern
    feature_columns=categorical_columns+continuous_columns,
    batch_size=BATCH_SIZE*4,
    label_names=LABEL_COLUMNS,
    shuffle=False,
    buffer_size=0.06 # amount of data, as a fraction of GPU memory, to load at once
)

### TensorFlow: Defining a Model

Using Keras, we can define the layers of our model and their parameters explicitly. Here, for the sake of consistency, we'll mimic fast.ai's [TabularModel](https://docs.fast.ai/tabular.learner.html).

In [12]:
# DenseFeatures layer needs a dictionary of {feature_name: input}
categorical_inputs = {}
for column_name in CATEGORICAL_COLUMNS:
    categorical_inputs[column_name] = tf.keras.Input(name=column_name, shape=(1,), dtype=tf.int64)
categorical_embedding_layer = tf.keras.layers.DenseFeatures(categorical_columns)
categorical_x = categorical_embedding_layer(categorical_inputs)
categorical_x = tf.keras.layers.Dropout(EMBEDDING_DROPOUT_RATE)(categorical_x)

# Just concatenating continuous, so can use a list
continuous_inputs = []
for column_name in CONTINUOUS_COLUMNS:
    continuous_inputs.append(tf.keras.Input(name=column_name, shape=(1,), dtype=tf.float32))
continuous_embedding_layer = tf.keras.layers.Concatenate(axis=1)
continuous_x = continuous_embedding_layer(continuous_inputs)
continuous_x = tf.keras.layers.BatchNormalization(epsilon=1e-5, momentum=0.1)(continuous_x)

# concatenate and build MLP
x = tf.keras.layers.Concatenate(axis=1)([categorical_x, continuous_x])
for dim, dropout_rate in zip(HIDDEN_DIMS, DROPOUT_RATES):
    x = tf.keras.layers.Dense(dim, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization(epsilon=1e-5, momentum=0.1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
x = tf.keras.layers.Dense(1, activation='linear')(x)

# TODO: Initialize model weights to fix saturation issues.
# For now, we'll just scale the output of our model directly before
# hitting the sigmoid.
x = 0.1 * x

x = MAX_LOG_SALES_PREDICTION * tf.keras.activations.sigmoid(x)

# combine all our inputs into a single list
# (note that you can still use .fit, .predict, etc. on a dict
# that maps input tensor names to input values)
inputs = list(categorical_inputs.values()) + continuous_inputs
tf_model = tf.keras.Model(inputs=inputs, outputs=x)

### TensorFlow: Training

In [13]:
def rmspe_tf(y_true, y_pred):
    # map back into "true" space by undoing transform
    y_true = tf.exp(y_true) - 1
    y_pred = tf.exp(y_pred) - 1

    percent_error = (y_true - y_pred) / y_true
    return tf.sqrt(tf.reduce_mean(percent_error**2))

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
tf_model.compile(optimizer, 'mse', metrics=[rmspe_tf])

In [15]:
%%time

optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
tf_model.compile(optimizer, 'mse', metrics=[rmspe_tf])

validation_callback = KerasSequenceValidater(valid_dataset_tf)
history = tf_model.fit(
    train_dataset_tf,
    callbacks=[validation_callback],
    epochs=EPOCHS,
)

Epoch 1/25
13/13 [==============================] - 4s 278ms/step - loss: 6.0261 - rmspe_tf: 0.8918 - val_loss: 6.7433 - val_rmspe_tf: 0.8898
Epoch 2/25
13/13 [==============================] - 3s 252ms/step - loss: 5.2557 - rmspe_tf: 0.8882 - val_loss: 6.1526 - val_rmspe_tf: 0.8884
Epoch 3/25
13/13 [==============================] - 4s 276ms/step - loss: 4.6124 - rmspe_tf: 0.8739 - val_loss: 5.7047 - val_rmspe_tf: 0.8730
Epoch 4/25
13/13 [==============================] - 3s 243ms/step - loss: 3.7993 - rmspe_tf: 0.8476 - val_loss: 5.1375 - val_rmspe_tf: 0.8461
Epoch 5/25
13/13 [==============================] - 3s 251ms/step - loss: 2.8152 - rmspe_tf: 0.8009 - val_loss: 4.4700 - val_rmspe_tf: 0.7983
Epoch 6/25
13/13 [==============================] - 3s 268ms/step - loss: 1.7985 - rmspe_tf: 0.7220 - val_loss: 3.8011 - val_rmspe_tf: 0.7183
Epoch 7/25
13/13 [==============================] - 3s 251ms/step - loss: 0.9462 - rmspe_tf: 0.6012 - val_loss: 3.2846 - val_rmspe_tf: 0.5937
Epoch 